# Ajuste de Curva

#### Imports

In [ ]:
from matplotlib.pyplot import rc, figure
from numpy import zeros, linspace
from numpy.typing import NDArray
from itertools import pairwise

#### Ajuste de curva

In [ ]:
def curve_adjust(D: NDArray, I: NDArray, n: int):
  G = D.reshape(-1, 1) ** range(n)
  A, B = (G.T @ G), (G.T @ I)

  L, U = zeros((n, n)), A
  L.flat[::(n + 1)] = 1

  for i, ii in pairwise(range(n)):
    L[ii:, i] = U[ii:, i] / U[i, i]
    U[ii:] -= L[ii:, i, None] * U[i]

  x, y = zeros(n), zeros(n)

  for i in range(n):
    y[i] = B[i] - (L[i] * y).sum()

  for i in reversed(range(n)):
    temp = y[i] - (U[i] * x).sum()
    x[i] = temp / U[i, i]

  return (G @ x)

#### Plotagem de gráficos

In [ ]:
def plotImg(D: NDArray, I: NDArray, R: NDArray):
  rc("font", family = "Arial", size = 10)
  rc("lines", aa = True, lw = .5)
  fig = figure(figsize = (12, 5))
  axe = fig.subplots(); fig.dpi = 500

  axe.plot(D, I, label = "Original")
  axe.plot(D, R, label = "Ajustado")
  axe.set_title("Ajuste de Curva")
  axe.set_xlabel("Domínio")
  axe.set_ylabel("Imagem")
  axe.legend()

  fig.tight_layout(pad = 1)

### Execução do Código

In [ ]:
def func(x: int | float | NDArray): return x**45
dom = linspace(-1, 1, 500); img = func(dom)
plotImg(dom, img, curve_adjust(dom, img, 15))